In [32]:
import tensorflow as tf


@tf.function(input_signature=[tf.TensorSpec(shape=[None, 2048, 1], dtype=tf.float32)])
def f(x):
    y = x + 2
    return y


f.input_signature

(TensorSpec(shape=(None, 2048, 1), dtype=tf.float32, name=None),)

In [33]:
class Model(tf.Module):

  @tf.function(input_signature=[tf.TensorSpec(shape=[None], dtype=tf.float32)])
  def encode(self, x):
    result = tf.strings.as_string(x)
    return {
         "encoded_result": result
    }

  @tf.function(input_signature=[tf.TensorSpec(shape=[None], dtype=tf.string)])
  def decode(self, x):
    result = tf.strings.to_number(x)
    return {
         "decoded_result": result
    }

In [34]:
model = Model()

converter = tf.lite.TFLiteConverter.from_concrete_functions([f.get_concrete_function()], model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/mb/z3d3qg1d4877v0lnbfy5wknc0000gp/T/tmpzzdvgbsi/assets


INFO:tensorflow:Assets written to: /var/folders/mb/z3d3qg1d4877v0lnbfy5wknc0000gp/T/tmpzzdvgbsi/assets
2023-02-17 01:16:23.200510: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-02-17 01:16:23.200527: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-02-17 01:16:23.200775: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/mb/z3d3qg1d4877v0lnbfy5wknc0000gp/T/tmpzzdvgbsi
2023-02-17 01:16:23.200942: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-02-17 01:16:23.200948: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/mb/z3d3qg1d4877v0lnbfy5wknc0000gp/T/tmpzzdvgbsi
2023-02-17 01:16:23.201683: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2023-02-17 01:16:23.201810: I tensorflow/cc/saved_model/load

In [35]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
signatures = interpreter.get_signature_list()
signatures

{'serving_default': {'inputs': ['x'], 'outputs': ['output_0']}}